# 1.1 MNIST dataset
## 1.1.1 Download MNIST data

In [1]:
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
# will download automatically when there is no data in "MNIST_data/"
%time mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
CPU times: user 521 ms, sys: 267 ms, total: 788 ms
Wall time: 3.95 s


In [3]:
# check size of training set
print(mnist.train.images.shape)
print(mnist.train.labels.shape)

(55000, 784)
(55000, 10)


In [4]:
# check size of validation set
print(mnist.validation.images.shape)
print(mnist.validation.labels.shape)

(5000, 784)
(5000, 10)


In [5]:
# check size of test set
print(mnist.test.images.shape)
print(mnist.test.labels.shape)

(10000, 784)
(10000, 10)


In [11]:
# check the 0th image
print(mnist.train.images[0, :20])

# check its label
print(mnist.train.labels[0, :])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]


## 1.1.2 Save MNIST dataset into images

In [13]:
import scipy.misc
import os

In [14]:
# folder to save images
save_dir = 'MNIST_data/images/'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [16]:
%%time
# save the top 20 images
for i in range(20):
    image_array = mnist.train.images[i, :]
    image_array = image_array.reshape(28, 28)
    
    filename = os.path.join(save_dir, 'mnist_train_%d.jpg' % i)
    scipy.misc.toimage(image_array, cmin=0.0, cmax=1.0).save(filename)

CPU times: user 28.2 ms, sys: 1.94 ms, total: 30.2 ms
Wall time: 111 ms


/home/saucecat/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use Pillow's ``Image.fromarray`` directly instead.
  import sys


## 1.1.3 Label one-hot encoding

In [17]:
import numpy as np

In [18]:
for i in range(20):
    one_hot_label = mnist.train.labels[i, :]
    label = np.argmax(one_hot_label)
    print('mnist_train_%d.jpg label: %d' %(i, label))

mnist_train_0.jpg label: 7
mnist_train_1.jpg label: 3
mnist_train_2.jpg label: 4
mnist_train_3.jpg label: 6
mnist_train_4.jpg label: 1
mnist_train_5.jpg label: 8
mnist_train_6.jpg label: 1
mnist_train_7.jpg label: 0
mnist_train_8.jpg label: 9
mnist_train_9.jpg label: 8
mnist_train_10.jpg label: 0
mnist_train_11.jpg label: 3
mnist_train_12.jpg label: 1
mnist_train_13.jpg label: 2
mnist_train_14.jpg label: 7
mnist_train_15.jpg label: 0
mnist_train_16.jpg label: 2
mnist_train_17.jpg label: 9
mnist_train_18.jpg label: 6
mnist_train_19.jpg label: 0


# 1.2 Classification task with TensorFlow

In [19]:
import tensorflow as tf

In [20]:
# x represents input images
# None means we don't know the batch size yet
x = tf.placeholder(dtype=tf.float32, shape=[None, 784])

# W represents trainable parameters, use Variable
W = tf.Variable(tf.zeros(shape=[784, 10]))
b = tf.Variable(tf.zeros(shape=[10]))

# y represents model output
probs = tf.nn.softmax(tf.matmul(x, W) + b)

# we need to specify the true label
# y need to be one-hot encoded
y = tf.placeholder(dtype=tf.float32, shape=[None, 10])

In [21]:
# calculate loss using probs and y
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y * tf.log(probs)))

# reduce the loss by optimizing parameters
train_step = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cross_entropy)

In [22]:
# create a session to train the model
sess = tf.InteractiveSession()

# initialize all parameters
tf.global_variables_initializer().run()
print('start training...')

for i in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(batch_size=100)
    sess.run(train_step, feed_dict={x: batch_xs, y: batch_ys})

start training...


In [23]:
# evaluate the model
correct_prediction = tf.equal(tf.argmax(probs, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print(sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels}))

0.9179


## 1.2.2 CNN

In [26]:
# some useful functions

# The generated values follow a normal distribution with 
# specified mean and standard deviation
# except that values whose magnitude is more than 2 standard deviations 
# from the mean are dropped and re-picked
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)
    
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)
    
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [38]:
# construct the graph
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])

x_image = tf.reshape(x, [-1, 28, 28, 1])

# padding = SAME
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])
y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [39]:
# define the loss
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=y_conv))

# define train step
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

# define accuracy
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
# create a session to train model
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

for i in range(20000):
    batch = mnist.train.next_batch(32)
    if i % 100 == 0:
        # when test accuracy, need to turn off the dropout
        train_accuracy = accuracy.eval(feed_dict={x: batch[0], y: batch[1], keep_prob: 1.0})
        print('step %d, training accuracy %g' %(i, train_accuracy))
    
    train_step.run(feed_dict={x: batch[0], y: batch[1], keep_prob: 0.5})
    
print('test accuracy %g' % accuracy.eval(feed_dict={x: mnist.test.images, y: mnist.test.labels, keep_prob: 1.0}))

## try to use tf.layers instead of tf.nn

In [45]:
# construct using tf.layers
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])

x_image = tf.reshape(x, [-1, 28, 28, 1])

h_conv1 = tf.layers.conv2d(inputs=x_image, filters=32, kernel_size=5, strides=(1, 1), padding='same', 
                           activation=tf.nn.relu, use_bias=True, 
                           kernel_initializer=tf.truncated_normal_initializer(stddev=0.1), 
                           bias_initializer=tf.constant_initializer(value=0.1))
h_pool1 = tf.layers.max_pooling2d(inputs=h_conv1, pool_size=2, strides=(2, 2), padding='same')

h_conv2 = tf.layers.conv2d(inputs=h_pool1, filters=64, kernel_size=5, strides=(1, 1), padding='same', 
                           activation=tf.nn.relu, use_bias=True, 
                           kernel_initializer=tf.truncated_normal_initializer(stddev=0.1), 
                           bias_initializer=tf.constant_initializer(value=0.1))
h_pool2 = tf.layers.max_pooling2d(inputs=h_conv2, pool_size=2, strides=(2, 2), padding='same')

h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
h_fc1 = tf.layers.dense(inputs=h_pool2_flat, units=1024, activation=tf.nn.relu, use_bias=True, 
                        kernel_initializer=tf.truncated_normal_initializer(stddev=0.1), 
                        bias_initializer=tf.constant_initializer(value=0.1))

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.layers.dropout(inputs=h_fc1, rate=1.-keep_prob)

y_conv = tf.layers.dense(inputs=h_fc1_drop, units=10, use_bias=True, 
                         kernel_initializer=tf.truncated_normal_initializer(stddev=0.1), 
                         bias_initializer=tf.constant_initializer(value=0.1))

In [46]:
# define the loss
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=y_conv))

# define train step
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

# define accuracy
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
# create a session to train model
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

for i in range(20000):
    batch = mnist.train.next_batch(32)
    if i % 100 == 0:
        # when test accuracy, need to turn off the dropout
        train_accuracy = accuracy.eval(feed_dict={x: batch[0], y: batch[1], keep_prob: 1.0})
        print('step %d, training accuracy %g' %(i, train_accuracy))
    
    train_step.run(feed_dict={x: batch[0], y: batch[1], keep_prob: 0.5})
    
print('test accuracy %g' % accuracy.eval(feed_dict={x: mnist.test.images, y: mnist.test.labels, keep_prob: 1.0}))

# More

- Official Website for MNIST: http://yann.lecun.com/exdb/mnist/
- TensorFlow official tutorial with MNIST: https://www.tensorflow.org/get_started/mnist/beginners and https://www.tensorflow.org/get_started/mnist/pros
- Tensor: https://www.tensorflow.org/programmers_guide/tensors
- Variables: https://www.tensorflow.org/programmers_guide/variables
- Session: https://www.tensorflow.org/programmers_guide/graphs